# Efficient Sorting

In this section, we will pull together everything we have learned in this algorithms module. **Merge Sort** is a parallelizable, divide and conquer algorithm which achieves the optimal time for general sorting algorithms.

## Sorting

We previously solved the sorting problem using Selection Sort. For every index in the list, this algorithm selects the element that belongs there and switches it into place. 

Here is the recursive implementation we gave to introduce recurrences.

In [ ]:
def selection_sort_recursive(L):
    print('L={}'.format(L))
    if (len(L) == 1):
        return(L)
    else:
        m = L.index(min(L))
        L[0], L[m] = L[m], L[0]
        return [L[0]] + selection_sort_recursive(L[1:])
    
selection_sort_recursive([2, 1, 999, 4, 3])

We analyzed its runtime, finding that the work done by selection sort is $W(n) \in \Theta(n^2)$.

What about its span? Well, since we only have a single recursive call at every step and each call depends on the next, selection sort isn't parallelizable. Its span is the same as its work.

#### Runtime Results for Selection Sort

$W(n) \in \Theta(n^2)$

$S(n) \in \Theta(n^2)$

## Applying Divide and Conquer to Search

When we studied the searching problem, we saw that we were able to gain some advantage by applying a divide and conquer approach.

### Linear Search

One strategy for solving the searching problem is to examine every element in turn:

In [1]:
def linear_search(lst, key):
    for element in lst:
        if element == key:
            return True
    return False

linear_search([2, 1, 999, 4, 3], 3)

True

Linear Search, as a sequential algorithm, has the same work and span:

$W(n) \in \Theta(n)$

$S(n) \in \Theta(n)$

### Divide and Conquer Search

An alternative strategy is to divide the list into two halves, search each half, and then combine the results to get the final answer. This is a classic divide and conquer approach.

Here is our divide and conquer `search_DC`:

In [2]:
def search_DC(lst, key):
    if len(lst) == 0:
        return False
    if len(lst) == 1:
        return lst[0] == key

    mid = len(lst)//2
    return search_DC(lst[:mid], key) or search_DC(lst[mid:], key)

lst = [4, 7, 2, 8, 1, 0, 3, 9, 5, 6]
search_DC(lst, 3)

True

Unlike linear search, divide and conquer search is parallelizable.

We analyzed `search_DC`'s work and span to be:

$W(n) \in \Theta(n)$

$S(n) \in \Theta(\log_2 n)$

## Applying Divide and Conquer to Sorting

Since we were able to gain an advantage by applying divide and conquer to searching, can we also do so for sorting?


### Algorithm Design

Starting with the whole list, we can divide it into two halves. If we then recursively sort each half, we can merge the sorted halves back together to get the fully sorted list.

Since this is a recursive algorithm what is the base case?

I think you will agree that a list of size $1$ is by definition sorted.

With this, we can sketch out our divide and conquer sorting algorithm. We haven't gotten to the details of merging sorted lists, so we will use `merge` as a placeholder function call for it. 

We'll call our sorting algorithm `merge_sort` since it relies on merging sorted lists. 

``` python
def merge_sort(lst):
    if len(lst) == 1:
        return lst
    
    mid = len(lst) // 2
    left = merge_sort(lst[:mid])
    right = merge_sort(lst[mid:])

    return merge(left, right)
```

## Will this Work?

How do we know that merge sort will correctly sort a list?

Well, once the recursion gets down to the base cases, `merge_sort` merges lists back together until it returns the whole list.

As long as `merge` returns a sorted list, every sucessive call to `merge` will take in two sorted lists and return a sorted list containing all elements in both. Since the base case returns a sorted list, merge will always have sorted lists as inputs and the final result of `merge_sort` will be the complete sorted list.

It helps to work through an example (I recommend this every time you study, learn, or design a new algorithm):


Here is how Merge Sort will operate on the list: $[4, 5, 2, 6, 1, 0, 3, 7]$.

You can trace through how Merge Sort divides the problem in half until it gets to the base cases, then merges the results back together until it gets to the final sorted list.

<img src = "figures/merge-sort.jpeg" width = "100%">

#### Merging Sorted Lists

We've so far left out how the `merge` algorithm works.

When `merge` is called, the `left` and `right` halves of the list have been sorted, so `merge`'s job is to merge together two sorted lists into a single sorted list.

To illustrate this algorithm, let's examine a concrete example:

Suppose we are merging the following two lists into a new sorted list.

$[\bm{5}, 7, 11]~~~[\bm{1}, 4, 6]$

Clearly the first element in the sorted list should be the smaller of the first two elements of the two lists, in this case: $1$.

`sorted_list`: $[1]$

Then, omitting it from consideration, the next element in the sorted list should be the smaller of $5$ and $4$, and so on.

$[\bm{5}, 7, 11]~~~[\bm{4}, 6]$

Adding $4$ to the sorted list.

`sorted_list`: $[1, 4]$

and so on...

At every step, we compare two elements, one from each list, and add the smaller to the end of the sorted list.

At some point, everything from one of the lists will have been added to the sorted list. At that point, everything else from the other list is added to the end.

# Complete Implementation

In [25]:
def merge_sort(lst):
    if len(lst) == 1:
        return lst
    
    mid = len(lst) // 2
    left = merge_sort(lst[:mid])
    right = merge_sort(lst[mid:])

    return merge(left, right, [])

def merge(left, right, sorted_lst=[]):
    if len(left) == 0:
        sorted_lst.extend(right)
        return sorted_lst
    if len(right) == 0:
        sorted_lst.extend(left)
        return sorted_lst

    if left[0] < right[0]:
        sorted_lst.append(left[0])
        return merge(left[1:], right, sorted_lst)
    else:
        sorted_lst.append(right[0])
        return merge(left, right[1:], sorted_lst)

merge_sort([4, 5, 2, 6, 1, 0, 3, 7])

[0, 1, 2, 3, 4, 5, 6, 7]

# Analysis

## Work of Merge Sort

We want to write down a recurrence for Merge Sort.

It is easy to see that each call to `merge_sort` generates two subproblems which are half of the size so we have the $a$ and $b$ for the general recurrence:

$$W(n) = 2W(n/2) + f(n)$$

But what is $f(n)$? The work within each `merge_sort` call is dominated by the work of `merge`. Everything else is constance time.



### Work of Merge

We can write down a recurrence for `merge`.

##### Merge Branching Factor

At every step, `merge` generates a single subproblem. Even though there are multiple recursive calls in our implementation, they are mutually exclusive and only one will be called.

##### Subproblem size

What about the size of the subproblem? At every step, we add a single element to the sorted list and then call `merge` on all the rest. The size of each subproblem is $n-1$.

##### Work of each call

Finally, how much work is done within each function call? The conditional checks and append calls are all constant time (let's ignore the extend call for now) so the work within each call is $O(1)$.


#### Solving the Recurrence

Our recurrence is $W(n) = W(n-1) + 1$

<img src = "figures/merge-work.jpeg" width = "100%">

This is trivial to solve.

$$
\begin{align}
W(n) & = \sum_{i=0}^{n-1} 1 \\
& = n \\
& \in \Theta(n)
\end{align}
$$

#### The extend Call

When considering the work within each call, we ignored the `extend` call. `extend` is not a constant time operation. Extend appends all elements passed in to it to the list. Its runtime is thus $O(n)$ depending on the number of elements being appended.

Does this break our analysis?

It does not. We can think about the runtime of `merge` as follows. Fundementally, `merge` is appending, one at a time, an element from each of the lists until both lists are empty. 

How many times will the append operation be executed? Clearly, if there are $n$ elements in both lists, there will be $n$ appends over the course of the algorithm. All other operations being constant time, the runtime of `merge` must be $\Theta(n)$

### Merge Sort's Recurrence

Finally, we have everything we need to write down Merge Sort's recurrence.

Every function call generates two subproblems, each half of the size, and requires within it linear work for merging.

$$W(n) = 2W(n/2) + n$$

We solved this recurrence previously in a video, but here its its tree again.

<img src = "figures/merge-sort-tree.jpeg" width = "100%">

It is a balanced tree, with a depth of $\log_2 n$.

$W(n) \in \Theta(n\log_2 n)$

## Span of Merge Sort

We can write down a the span of Merge Sort as follows:

$$S(n) = S(n/2) + n$$

For this recurrence, the branching factor is $1$. Why? In analyzing the span, we assume that all recursive calls on a level happen in parallel. We can ignore the multiple nodes on each level of the full recursion tree for Merge Sort.

For the division factor, each recursive call still operates on problems of half the size, so it is still $2$.

Why do we have a linear cost for the span within each function call?

Our `merge` algorithm is sequential. Every step depends on the previous so the span of `merge` is $O(n)$, the same as its work.

Let's solve for the span:

<img src = "figures/merge-sort-span.jpeg" width = "100%">

The work is geometrically decreasing by a factor of $1/4$.

This tree is **root dominated**.

$S(n) \in \Theta(n)$

## Summary

For Merge Sort, we have the following work and span:

**Work:** $W(n) \in \Theta(n\log_2 n)$

**Span:** $S(n) \in \Theta(n)$

Compared against Selection Sort:

**Work:** $W(n) \in \Theta(n^2)$

**Span:** $S(n) \in \Theta(n^2)$

This is much better!

By applying a divide and conquer strategy to our sorting algorithm, we were able to achieve a significant improvement in the runtime of our algorithms!

### Epiloque: Parallel Merge

Even though we haven't discussed it, there does exist a parallel algorithm for the merge operation of Merge Sort.

The span of parallel merge is $\Theta(\log_2 n)$.

If we use this parallel merge, then the span of Merge Sort can be expressed as:

$$S(n) = S(n/2) + \log_2 n$$

You solved this recurrence in lab-04. If you are reading ahead and haven't solved it yet, then you will note that it doesn't follow a geometric pattern. You will have to use the recursion tree method to solve it.

This recurrence will require some algebra and you will find our [Logs, Exponents, and Series Reference](reference/logs-exponents-series.pdf) useful. It is great practice for your recurrence solving skills!

# Lagniappe: Optimal Sorting

It turns out that not only does Merge Sort achieve a better runtime than other sorting algorithms, but it achieves the optimal runtime.

Merge Sort achieves the best possible runtime for any general sorting algorithm.

Specifically, the runtime of any algorithm which sorts elements by comparing elements is:

$$\Omega(n \log_2 n)$$

It's not that we aren't clever enough to invent a more efficient algorithm, it's instead that it is impossible to do so.

This sort of result is pretty amazing and proofs which can make statements like this are worth studying.

For your interest, here is a reference which consisely gives that proof: [Sorting Lower Bound](reference/optimal-sorting-proof.pdf)